In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn import preprocessing
import itertools 
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from sklearn import datasets
from keras.layers import Activation, Dense


In [2]:
data = pd.read_csv("normal_data.csv")
data.head()

,Time,DayOfWeek,Motion,Acceleration,Temperature
0,13:53:54.641,4,inactive,inactive,73
1,14:54:9.094,0,inactive,inactive,62
2,09:29:43.859,3,inactive,inactive,67
3,12:07:47.004,0,inactive,active,68
4,09:20:4.064,2,active,inactive,60


### Sort the data by time
#### first by DayOfWeek, then by Time

In [3]:
data_sorted = data.sort_values(by=['DayOfWeek', 'Time'])
data_sorted

,Time,DayOfWeek,Motion,Acceleration,Temperature
4413,00:04:53.008,0,inactive,inactive,69
3913,00:23:1.685,0,active,inactive,66
2149,00:30:34.812,0,inactive,active,62
3970,00:33:3.557,0,inactive,inactive,62
9009,00:39:29.942,0,inactive,active,63
3248,00:41:12.704,0,inactive,inactive,69
2729,00:48:11.16,0,inactive,inactive,66
6425,00:48:30.949,0,active,inactive,74
4339,00:48:6.632,0,inactive,active,63
2411,00:50:7.424,0,active,inactive,69


In [4]:
(data_sorted[['Temperature']]).max()

Temperature    74
dtype: int64

In [5]:
(data_sorted[['Temperature']]).min()

Temperature    60
dtype: int64

### 56 possible distinct log keys (14 * 4)

In [6]:
data_sorted.groupby(['Motion', 'Acceleration', 'Temperature']).size()

Motion    Acceleration  Temperature
active    active        60              29
                        61              27
                        62              26
                        63              27
                        64              27
                        65              29
                        66              24
                        67              30
                        68              31
                        69              21
                        70              22
                        71              24
                        72              25
                        73              19
                        74              23
          inactive      60             103
                        61              97
                        62             105
                        63             107
                        64             106
                        65             101
                        66             107
                  

### pre-processing the input data for LSTM 


In [7]:
data_input = data_sorted.drop(['Time', 'DayOfWeek'], axis = 1)
data_input.head()

,Motion,Acceleration,Temperature
4413,inactive,inactive,69
3913,active,inactive,66
2149,inactive,active,62
3970,inactive,inactive,62
9009,inactive,active,63


In [8]:
le = preprocessing.LabelEncoder()
data_input = data_input.apply(le.fit_transform)
data_input

,Motion,Acceleration,Temperature
4413,1,1,9
3913,0,1,6
2149,1,0,2
3970,1,1,2
9009,1,0,3
3248,1,1,9
2729,1,1,6
6425,0,1,14
4339,1,0,3
2411,0,1,9


### assign the class for each sample of logs

In [9]:
l=[ [0, 1], [0, 1] ]
temp = []
for i in range(1, 15):
    temp.append(i)
l.append(temp)

lookup_list = list(itertools.product(*l)) 
lookup_list

[(0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 0, 4),
 (0, 0, 5),
 (0, 0, 6),
 (0, 0, 7),
 (0, 0, 8),
 (0, 0, 9),
 (0, 0, 10),
 (0, 0, 11),
 (0, 0, 12),
 (0, 0, 13),
 (0, 0, 14),
 (0, 1, 1),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 1, 5),
 (0, 1, 6),
 (0, 1, 7),
 (0, 1, 8),
 (0, 1, 9),
 (0, 1, 10),
 (0, 1, 11),
 (0, 1, 12),
 (0, 1, 13),
 (0, 1, 14),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 3),
 (1, 0, 4),
 (1, 0, 5),
 (1, 0, 6),
 (1, 0, 7),
 (1, 0, 8),
 (1, 0, 9),
 (1, 0, 10),
 (1, 0, 11),
 (1, 0, 12),
 (1, 0, 13),
 (1, 0, 14),
 (1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 1, 4),
 (1, 1, 5),
 (1, 1, 6),
 (1, 1, 7),
 (1, 1, 8),
 (1, 1, 9),
 (1, 1, 10),
 (1, 1, 11),
 (1, 1, 12),
 (1, 1, 13),
 (1, 1, 14)]

In [10]:
data_input = data_input.reset_index(drop=True)
data_input

,Motion,Acceleration,Temperature
0,1,1,9
1,0,1,6
2,1,0,2
3,1,1,2
4,1,0,3
5,1,1,9
6,1,1,6
7,0,1,14
8,1,0,3
9,0,1,9


In [11]:
category = []
for i in range(10000):
    category.append(0)
len(category )

10000

In [12]:

for index, row in data_input.iterrows():
    for i in range(56):
        if (lookup_list[i][0] == row['Motion'] and lookup_list[i][1] == row['Acceleration'] 
            and lookup_list[i][2] == row['Temperature']):
            category[index] = i
               

In [13]:
category

[50,
 19,
 29,
 43,
 30,
 50,
 47,
 27,
 30,
 22,
 52,
 54,
 48,
 31,
 39,
 46,
 47,
 47,
 30,
 53,
 38,
 42,
 29,
 50,
 31,
 49,
 54,
 52,
 17,
 29,
 47,
 21,
 49,
 48,
 34,
 51,
 33,
 54,
 52,
 54,
 21,
 46,
 50,
 49,
 42,
 44,
 51,
 24,
 50,
 29,
 21,
 38,
 50,
 32,
 55,
 0,
 46,
 46,
 41,
 43,
 49,
 40,
 50,
 41,
 52,
 50,
 54,
 47,
 46,
 49,
 49,
 46,
 42,
 14,
 46,
 0,
 30,
 20,
 51,
 55,
 49,
 43,
 53,
 55,
 0,
 20,
 46,
 52,
 0,
 54,
 44,
 45,
 53,
 42,
 0,
 50,
 39,
 54,
 53,
 0,
 45,
 54,
 49,
 9,
 54,
 0,
 32,
 45,
 45,
 44,
 14,
 44,
 0,
 50,
 0,
 49,
 0,
 45,
 44,
 54,
 5,
 0,
 23,
 0,
 17,
 55,
 29,
 52,
 49,
 45,
 38,
 43,
 36,
 45,
 20,
 46,
 55,
 47,
 50,
 16,
 43,
 18,
 37,
 0,
 43,
 44,
 22,
 32,
 55,
 4,
 19,
 43,
 45,
 44,
 48,
 36,
 40,
 52,
 50,
 17,
 51,
 43,
 14,
 51,
 25,
 48,
 49,
 24,
 53,
 34,
 52,
 14,
 54,
 44,
 49,
 46,
 48,
 36,
 48,
 42,
 48,
 23,
 41,
 49,
 53,
 49,
 50,
 0,
 44,
 33,
 16,
 45,
 34,
 21,
 45,
 52,
 44,
 46,
 32,
 20,
 54,
 37,
 25,
 3

In [14]:

ct = ColumnTransformer([
        ('somename', StandardScaler(), ['Temperature'])
    ], remainder='passthrough')

data_input_2 = ct.fit_transform(data_input)
data_input_2[0:20]


/Users/ShenSiyuan/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/ShenSiyuan/anaconda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([[ 0.46943238,  1.        ,  1.        ],
       [-0.22891987,  0.        ,  1.        ],
       [-1.16005619,  1.        ,  0.        ],
       [-1.16005619,  1.        ,  1.        ],
       [-0.92727211,  1.        ,  0.        ],
       [ 0.46943238,  1.        ,  1.        ],
       [-0.22891987,  1.        ,  1.        ],
       [ 1.63335278,  0.        ,  1.        ],
       [-0.92727211,  1.        ,  0.        ],
       [ 0.46943238,  0.        ,  1.        ],
       [ 0.93500054,  1.        ,  1.        ],
       [ 1.4005687 ,  1.        ,  1.        ],
       [ 0.00386422,  1.        ,  1.        ],
       [-0.69448803,  1.        ,  0.        ],
       [ 1.16778462,  1.        ,  0.        ],
       [-0.46170395,  1.        ,  1.        ],
       [-0.22891987,  1.        ,  1.        ],
       [-0.22891987,  1.        ,  1.        ],
       [-0.92727211,  1.        ,  0.        ],
       [ 1.16778462,  1.        ,  1.        ]])

In [15]:
data_input_2.shape

(10000, 3)

### method II for labels

### assume 50 time steps

In [16]:
Y_labels = []
X = []
for i in range(49, 9999):
    temp = []
    for j in range(50):
        temp.append(data_input_2[i-j])
    X.append(temp)
    Y_labels.append(category[i+1])
        
        
    

In [17]:
X = np.asarray(X)

In [18]:
X

array([[[-1.16005619,  1.        ,  0.        ],
        [ 0.46943238,  1.        ,  1.        ],
        [ 0.93500054,  0.        ,  1.        ],
        ..., 
        [-1.16005619,  1.        ,  0.        ],
        [-0.22891987,  0.        ,  1.        ],
        [ 0.46943238,  1.        ,  1.        ]],

       [[ 0.2366483 ,  0.        ,  1.        ],
        [-1.16005619,  1.        ,  0.        ],
        [ 0.46943238,  1.        ,  1.        ],
        ..., 
        [-1.16005619,  1.        ,  1.        ],
        [-1.16005619,  1.        ,  0.        ],
        [-0.22891987,  0.        ,  1.        ]],

       [[ 0.93500054,  1.        ,  0.        ],
        [ 0.2366483 ,  0.        ,  1.        ],
        [-1.16005619,  1.        ,  0.        ],
        ..., 
        [-0.92727211,  1.        ,  0.        ],
        [-1.16005619,  1.        ,  1.        ],
        [-1.16005619,  1.        ,  0.        ]],

       ..., 
       [[-1.16005619,  1.        ,  1.        ],
        

### Check the size of the features and the labels

In [60]:
X.shape

(9950, 50, 3)

In [20]:
len(Y_labels )

9950

### Train the LSTM model 
#### add LSTM with input shape (50,3): 50 time steps and 3 features

In [61]:
model = Sequential()
model.add(LSTM(56, input_shape=(50, 3)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

ValueError: All layers in a Sequential model should have a single output tensor. For multi-output layers, use the functional API.

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_labels, test_size=0.3, random_state=0)

In [59]:
X_train.shape, len(y_train)

((6965, 50, 3), 6965)

In [57]:
history = model.fit(X_train, y_train, epochs=5, batch_size=10, verbose=2, shuffle=False)
history

ValueError: Error when checking target: expected activation_7 to have shape (56,) but got array with shape (1,)

In [25]:
accr = model.evaluate(X_test,y_test)
accr

2985/2985 [==============================] - 2s 732us/step


[-580.90473978371676, 0.002680067001675042]

In [30]:
prediction = model.predict(X_test, verbose=0)
prediction.tolist()

[[1.6690466403961182],
 [1.784240961074829],
 [1.7642616033554077],
 [1.688876986503601],
 [1.7237356901168823],
 [1.8271127939224243],
 [1.575282335281372],
 [1.920105218887329],
 [1.3404293060302734],
 [1.7342694997787476],
 [1.560116171836853],
 [1.8649003505706787],
 [1.8940017223358154],
 [1.884795069694519],
 [1.7167054414749146],
 [1.6504348516464233],
 [1.9204636812210083],
 [1.6417741775512695],
 [1.8410842418670654],
 [1.8280892372131348],
 [1.6574350595474243],
 [1.6746996641159058],
 [1.834212064743042],
 [1.7959480285644531],
 [1.7014782428741455],
 [1.7059788703918457],
 [1.7399381399154663],
 [1.8664419651031494],
 [1.8851633071899414],
 [1.7127137184143066],
 [1.6970036029815674],
 [1.6570289134979248],
 [1.5352094173431396],
 [1.9096260070800781],
 [1.8275554180145264],
 [1.6597673892974854],
 [1.8800400495529175],
 [1.7153337001800537],
 [1.5873055458068848],
 [1.583439826965332],
 [1.7292382717132568],
 [1.3734087944030762],
 [1.7531483173370361],
 [1.733957409858703

In [31]:
prediction.shape

(2985, 1)

In [29]:
y_test

[47,
 47,
 49,
 28,
 37,
 46,
 53,
 33,
 42,
 50,
 55,
 45,
 41,
 42,
 47,
 36,
 16,
 16,
 48,
 55,
 55,
 52,
 25,
 52,
 49,
 45,
 11,
 52,
 0,
 44,
 54,
 51,
 0,
 46,
 47,
 54,
 34,
 49,
 55,
 50,
 49,
 42,
 52,
 49,
 51,
 36,
 48,
 55,
 45,
 22,
 14,
 26,
 48,
 12,
 0,
 0,
 16,
 31,
 46,
 43,
 49,
 10,
 26,
 23,
 45,
 0,
 50,
 51,
 52,
 24,
 45,
 51,
 3,
 41,
 34,
 55,
 50,
 48,
 53,
 0,
 26,
 42,
 45,
 51,
 53,
 55,
 55,
 19,
 55,
 0,
 51,
 4,
 47,
 47,
 51,
 26,
 18,
 21,
 11,
 44,
 26,
 50,
 20,
 39,
 45,
 55,
 50,
 39,
 38,
 47,
 50,
 30,
 11,
 48,
 42,
 55,
 44,
 43,
 54,
 50,
 43,
 0,
 55,
 36,
 46,
 55,
 53,
 52,
 4,
 18,
 42,
 48,
 52,
 35,
 47,
 30,
 49,
 54,
 46,
 55,
 46,
 27,
 48,
 46,
 45,
 48,
 47,
 52,
 42,
 36,
 53,
 47,
 0,
 43,
 34,
 50,
 21,
 54,
 45,
 37,
 30,
 50,
 55,
 22,
 52,
 46,
 47,
 47,
 51,
 50,
 17,
 52,
 10,
 17,
 45,
 53,
 48,
 43,
 30,
 44,
 51,
 42,
 38,
 46,
 31,
 31,
 50,
 0,
 44,
 42,
 51,
 55,
 44,
 48,
 44,
 47,
 26,
 29,
 54,
 49,
 55,
 46,
 52

In [33]:
len(y_test)

2985

# Method II


In [65]:
for i in range(len(y_test)):
    y_test[i] = 1

for i in range(len(y_train)):
    y_train[i] = 1 
    
y_train

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [68]:
model = Sequential()
model.add(LSTM(100, input_shape=(50, 3)))
model.add(Dense(1, activation = 'softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 100)               41600     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 41,701
Trainable params: 41,701
Non-trainable params: 0
_________________________________________________________________
None


In [69]:
history = model.fit(X_train, y_train, epochs=5, batch_size=10, verbose=2, shuffle=False)
history

Epoch 1/5
 - 116s - loss: 1.1921e-07 - acc: 1.0000
Epoch 2/5
 - 97s - loss: 1.1921e-07 - acc: 1.0000
Epoch 3/5
 - 88s - loss: 1.1921e-07 - acc: 1.0000
Epoch 4/5
 - 86s - loss: 1.1921e-07 - acc: 1.0000
Epoch 5/5
 - 85s - loss: 1.1921e-07 - acc: 1.0000
